In [29]:
import numpy as np
import statsmodels.formula.api as smf
from numpy.linalg import inv
import pandas as pd
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn.linear_model import LogisticRegression as Log_Reg
from statsmodels.regression.linear_model import OLS
from sklearn.cluster import KMeans
import sklearn.preprocessing as Preprocessing
from sklearn.preprocessing import StandardScaler as Standardize
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
from itertools import combinations
%matplotlib inline

# Modeling: 
<span style='Color:purple'> Now, that we have two clean datasets, we can start the modeling. </span>

## Uploading Data
<span style='Color:purple'> Let's upload our datasets and merge them: </span>

In [2]:
#Import data
election_data_all = pd.read_csv('election_data.csv')

#Create a column with elkection year
election_data_all['RaceYear'] = pd.Series(map(lambda x : int(str(x)[:4]), list(election_data_all['RaceDate'])))

<span style='Color:purple'> We choose to keep the election year and the republican Major % vote and state. The republican % major vote is chosen as a proxy becaut it does not tske into account third parties or independent candidates. </span>

In [3]:
election_data = election_data_all[['RaceYear', 'Area', 'RepVotesMajorPercent']]

In [4]:
election_data['RaceYear'].unique()

array([1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996,
       2000, 2004, 2008, 2012])

In [5]:
#Import yearly name pca for each state and concatenate
names_pca_states = pd.DataFrame()
for year in xrange(1910, 2013):
    dataframe = pd.read_csv('matrices_and_pca/%d_data_pca.csv' % year)
    dataframe.columns = [('%d_PCA1' % year),('%d_PCA2' % year)]
    names_pca_states = pd.concat([names_pca_states, dataframe], axis=1)

In [6]:
#Map states in both files before merging
original_pairs = pd.read_csv('us_states_reordered.csv',header=None)
del original_pairs[2]

original_pairs.columns = ['Name','Abb']
my_dict = original_pairs.set_index('Name')['Abb'].to_dict()

election_data = election_data.replace({"Area":my_dict})

names_pca_states.index = original_pairs.Abb.values

<span style='Color:purple'> A way to look at this is for each election look at teh relationship between the % of biblical names in the years all the range of the voters (18-75) can predict the political vote. Since we only have baby names from 1910, for this analysis we can only look at elections 1988 onwards. </span>

In [7]:
#Merge datasets
election_names_states = election_data[election_data['RaceYear'] >= 1988]

In [8]:
#Merge datasets
election_names_states = election_data[election_data['RaceYear'] >= 1988]
holder = []
ctr = 0 
for key,grp in election_names_states.groupby('RaceYear'):

    tmp_votes = grp
    tmp_votes.index = grp.Area

    year_min=int(key)-75
    year_max = int(key)-18

    tmp_vals = names_pca_states.ix[:,('%d_PCA1' % year_min) : ('%d_PCA2' % year_max)]
    tmp_vals = tmp_vals.merge(tmp_votes, left_index=True, right_index=True)
    tmp_vals.columns= range(0,len(tmp_vals.columns))

    if (ctr ==0):
        master_df=tmp_vals
        ctr = ctr+1
    else:
        frms = [master_df,tmp_vals]
        master_df = pd.concat(frms)

master_df = master_df.rename(columns={116:'year', 117:'state', 118 : 'y'})
t = master_df['y'] >= 50
master_df['t'] = t


In [9]:
#Assign x and y 
y = master_df['y']
x = master_df[range(0,116)]

In [10]:
# ### Step-wise Forward Selection
# d = x.shape[1] # total no. of predictors

# # Keep track of current set of chosen predictors, and the remaining set of predictors
# current_predictors = [] 
# remaining_predictors = range(d)

# # Set some initial large value for min BIC score for all possible subsets
# global_min_bic = 1e10 

# # Keep track of the best subset of predictors
# best_subset = [] 

# # Iterate over all possible subset sizes, 0 predictors to d predictors
# for size in range(d):    
#     max_r_squared = -1e10 # set some initial small value for max R^2
#     best_predictor = -1 # set some throwaway initial number for the best predictor to add
#     bic_with_best_predictor = 1e10 # set some initial large value for BIC score   
        
#     # Iterate over all remaining predictors to find best predictor to add
#     for i in remaining_predictors:
#         # Make copy of current set of predictors
#         temp = current_predictors[:]
        
#         # Add predictor 'i'
#         temp.append(i)
        
#         # Use only a subset of predictors in the training data
#         x_subset = x[:, temp]
#         # Add a column of ones
#         x_subset = np.hstack((x_subset, np.ones((x_subset.shape[0], 1))))
        
#         # Fit and evaluate R^2
#         model = OLS(y, x_subset)
#         results = model.fit()
#         r_squared = results.rsquared
        
#         # Check if we get a higher R^2 value than than current max R^2, if so, update
#         if(r_squared > max_r_squared):
#             max_r_squared = r_squared
#             best_predictor = i
#             bic_with_best_predictor = results.bic
    
#     # Remove best predictor from remaining list, and add best predictor to current list
#     remaining_predictors.remove(best_predictor)
#     current_predictors.append(best_predictor)
    
#     # Check if BIC for with the predictor we just added is lower than 
#     # the global minimum across all subset of predictors
#     if(bic_with_best_predictor < global_min_bic):
#         best_subset = current_predictors[:]
#         global_min_bic = bic_with_best_predictor
    
# print 'Step-wise forward subset selection:'
# print sorted(best_subset) # add 1 as indices start from 0

In [11]:
# Lasso regression
lasso_reg = Lasso_Reg(alpha = 0.01)
lasso_reg.fit(x, y)
score_lasso = lasso_reg.score(x,y)

print 'Lasso score is :', score_lasso

Lasso score is : 0.252190723433


In [12]:
# Standardize data
x_std = Standardize(with_mean=False).fit_transform(x)

# Ridge regression: Fit and evaluate 
ridge_reg = Ridge_Reg(alpha = 0.01)
ridge_reg.fit(x_std, y)
score_ridge = ridge_reg.score(x,y)

print 'Ridge score is :', score_ridge

Ridge score is : 0.0118787512603


In [13]:
# # Rewrite with MSE instead of R-squared
# # (as the labels have a lot of zeros, hence r_squared can blow up)
# def k_fold_perf_mse(x, y, n_folds, param_val):
#     n_train = np.shape(x_train)[0]
#     n = int(np.round(n_train * 1. / n_folds)) # points per fold
    
#     # Iterate over folds
#     cv_mse = 0
#     for fold in range(1, n_folds + 1):
#         x_train_cv = np.concatenate((x_train[:n * (fold - 1), :], x_train[n * fold + 1:, :]), axis=0)
#         y_train_cv = np.concatenate((y_train[:n * (fold - 1)], y_train[n * fold + 1:]), axis=0)

#         x_test_cv = x_train[1 + n * (fold - 1):n * fold, :]
#         y_test_cv = y_train[1 + n * (fold - 1):n * fold]

#         # Fit ridge regression model with parameter value on CV train set
#         reg = Ridge_Reg(alpha = param_val)
#         reg.fit(x_train_cv, y_train_cv)
        
#         # Evaluate mean squared error of model on CV test set
#         y_pred_cv = reg.predict(x_test_cv)
#         mse = np.mean(np.square(y_test_cv - y_pred_cv))

#         cv_mse += mse

#     return cv_mse * 1.0 / n_folds


# # CV to tune parameter in Ridge regression
# min_mse = 1e10 # some high value
# min_param = -1

# for power_of_10 in range(-7, 7):
#     cv_mse = k_fold_perf_mse(x, y, 5, 10.0**power_of_10)
    
#     if(cv_mse < min_mse):
#         min_mse = cv_mse
#         min_param = 10**power_of_10
    
# # Fit regression model with opt parameter on train set
# reg = Ridge_Reg(alpha = min_param)
# reg.fit(x_train, y_train)

# # Evaluate R-squared on train and test sets
# train_r_squared = reg.score(x_train, y_train)
# test_r_squared = reg.score(x_test, y_test)

# print 'Ridge regression'

# print "Train R-squared:", train_r_squared 
# print "Test R-squared:", test_r_squared

# # Evaluate profit on test set
# y_pred = reg.predict(x_test)

# doners = (y_pred - cost_per_donor) > 0
# profit = (y_test[doners] - cost_per_donor).sum()

# print "Total donations:", profit

In [14]:
#from http://planspace.org/20150423-forward_selection_with_statsmodels/
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove('y')
    remaining.remove('t')
    remaining.remove('year')
    remaining.remove('state')
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [15]:
#from http://planspace.org/20150423-forward_selection_with_statsmodels/

master_df.columns = map(lambda x : 'x' + str(x) if type(x) != str else x, master_df.columns)
model = forward_selected(master_df, 'y')

print model.model.formula

print model.rsquared_adj

y ~ x64 + x112 + x17 + x2 + x71 + x98 + x61 + x77 + x50 + x74 + x105 + x59 + x7 + x13 + x33 + x29 + x51 + x100 + x41 + x21 + x65 + x30 + x26 + x37 + x45 + x9 + x10 + x91 + x36 + x46 + x35 + x75 + x53 + x93 + x54 + x22 + x6 + x69 + x78 + x25 + x114 + x90 + x80 + x82 + x70 + x73 + x38 + x113 + 1
0.522726389863


In [58]:
pred_cols = model.model.formula.split(' + ')[1:-1]

X = master_df[pred_cols]
t = master_df['t']

print Ridge_Reg().fit(X, y).score(X, y)

0.0671273487026
